In [1]:

import osmnx as ox
import folium
import pandas as pd
import numpy as np

from IPython.display import display
pd.set_option('display.max_rows', None)

# 峠道
graph = ox.graph_from_point(center_point=(35.2894024,136.9238574)
                                , network_type='drive'
                                , dist=5000
                                , simplify=True
                                , custom_filter='["highway"~"tertiary|secondary|primary|trunk"]')

# グラフデータをGeoDataFrameに変換  
gdf_nodes = ox.graph_to_gdfs(graph, nodes=True, edges=False)
gdf_edges = ox.graph_to_gdfs(graph, nodes=False, edges=True)

# エッジの長さから評価する。
lower_bound_meter = 200
upper_bound_meter = 2000
gdf_edges['length_evaluation'] = np.where(gdf_edges['length'] <= lower_bound_meter, 0,
                                   np.where(gdf_edges['length'] >= upper_bound_meter, 1,
                                            (gdf_edges['length'] - lower_bound_meter) / (upper_bound_meter - lower_bound_meter)))

# エッジからジオメトリーのポイント数を取得する(ポイントが多い = Rがついている道?, エッジが重なっている所もポイント数が多くなるので注意(後々で別の評価関数を実装する予定))
gdf_edges['point_cnt'] = gdf_edges['geometry'].apply(lambda x: len(x.coords))

# エッジの距離とポイント数から曲線率?を求める。
gdf_edges['curvature'] = gdf_edges['point_cnt'] / gdf_edges['length']

G = ox.graph_from_gdfs(gdf_nodes, gdf_edges)

def get_nearest_node_by_coordinates(graph, latitude, longitude):
    nearest_node = ox.distance.nearest_nodes(
        graph, longitude, latitude, return_dist=False
    )
    return nearest_node

hakatasta_node = get_nearest_node_by_coordinates(G, 35.334446, 136.990590)

def get_coordinates_by_node_id(graph, node_id):
    node_data = graph.nodes[node_id]
    x = node_data['x']
    y = node_data['y']
    return x, y

a = get_coordinates_by_node_id(G, hakatasta_node)
node_colors = ['red' if node == hakatasta_node else 'blue' for node in graph.nodes]
# ox.plot_graph(G, node_color=node_colors)

# 地図を表示する
map_osm = ox.plot_graph_folium(ox.utils_graph.get_largest_component(G, strongly=True), edge_width=2)

# 35.2894024,136.9238574にplotする
folium.Marker(
    location=[35.2894024,136.9238574],
    popup="st",
    icon=folium.Icon(color='green', icon='ok-sign'),
).add_to(map_osm)

# 曲線率が0.025より上 かつ length_evaluationが0.5以上のエッジにplotを立てて緯度と経度を表示する
for _, row in gdf_edges[(gdf_edges['curvature'] > 0.025) & (gdf_edges['length_evaluation'] > 0.5)].iterrows():
    folium.Marker(
        location=[row['geometry'].coords[0][1], row['geometry'].coords[0][0]],
        popup=f"{row['geometry'].coords[0][1]}, {row['geometry'].coords[0][0]}",
        icon=folium.Icon(color='red', icon='ok-sign'),
    ).add_to(map_osm)

# 評価値を元にエッジのスタイルを変える
map_osm.add_child(
    folium.features.GeoJson(
        gdf_edges[gdf_edges['length_evaluation'] > 0.5].to_json(
            ),
        # https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html
        style_function=lambda x: {
            'color': '#ff0000',
            'fill_opacity': 0,
            # 曲線率が0.025より上のものは太くする
            'weight': 5 if x['properties']['curvature'] > 0.025 else 1,
            # 'opacity': x['properties']['length_evaluation'] * ,
            # 'weight': 5 * x['properties']['length_evaluation'],
        }
    )
)


/tmp/ipykernel_2776/87173759.py:54: UserWarning: The `folium` module has been deprecated and will be removed in a future release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  map_osm = ox.plot_graph_folium(ox.utils_graph.get_largest_component(G, strongly=True), edge_width=2)


NameError: name 'row' is not defined